In [1]:
# !wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/news/fake-news/indonesian/250%20news%20with%20valid%20hoax%20label.csv
# !wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/news/fake-news/indonesian/600%20news%20with%20valid%20hoax%20label.csv
# !wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/news/fake-news/indonesian/facts.csv
# !wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/news/fake-news/indonesian/hoax.csv

In [2]:
import json
import os
import glob

In [3]:
negatives = glob.glob('/home/husein/Malaya-Dataset/fake-news/negative/*.json')
positives = glob.glob('/home/husein/Malaya-Dataset/fake-news/positive/*.json')
negatives

['/home/husein/Malaya-Dataset/fake-news/negative/clean-gossipcop_real2.json',
 '/home/husein/Malaya-Dataset/fake-news/negative/clean-gossipcop_real3.json',
 '/home/husein/Malaya-Dataset/fake-news/negative/1.json',
 '/home/husein/Malaya-Dataset/fake-news/negative/clean-gossipcop_real1.json',
 '/home/husein/Malaya-Dataset/fake-news/negative/clean-politifact_real.json',
 '/home/husein/Malaya-Dataset/fake-news/negative/2.json']

In [4]:
texts, labels = [], []
for negative in negatives:
    with open(negative) as fopen:
        x = json.load(fopen)
    texts.extend(x)
    labels.extend([1] * len(x))

In [5]:
for positive in positives:
    with open(positive) as fopen:
        x = json.load(fopen)
    texts.extend(x)
    labels.extend([0] * len(x))

In [6]:
import numpy as np
import pandas as pd

In [7]:
np.unique(labels, return_counts = True)

(array([0, 1]), array([15191, 26832]))

In [8]:
df = pd.read_csv('Malaya-Dataset/fake-news/malaysia-scraping-syazanihussin.csv').dropna()
df.head()

,News,Label
0,"Produk Kod E Mengandungi Lemak Babi. Awas, jan...",Fake
1,Jabatan Kemajuan Islam Malaysia memperjelaskan...,Real
2,Roti Massimo Mengandungi DNA Babi. Roti produk...,Fake
3,Jabatan Kemajuan Islam Malaysia (JAKIM) melalu...,Real
4,Tiada Pembayaran Pencen Selepas Persaraan 2021...,Fake


In [9]:
df_fake = df.loc[df['Label'] == 'Fake']
df_real = df.loc[df['Label'] == 'Real']
df_fake.shape, df_real.shape

((239, 2), (232, 2))

In [10]:
df_250 = pd.read_csv('250 news with valid hoax label.csv', sep = ';', encoding = "ISO-8859-1")
df_600 = pd.read_csv('600 news with valid hoax label.csv', sep = ';', encoding = "ISO-8859-1")

In [11]:
facts = pd.read_csv('facts.csv')
hoax = pd.read_csv('hoax.csv')

In [13]:
fact_isi = facts['Isi'].tolist()

hoax_judul = [i for i in hoax['Judul'].tolist() if len(i) > 25]
hoax_isi = hoax['Isi'].tolist()

In [14]:
mapping = {'Valid': 1, 'Hoax': 0}

labels.extend(df_250['tagging'].map(mapping).tolist())
labels.extend(df_600['tagging'].map(mapping).tolist())
texts.extend(df_250['berita'].tolist())
texts.extend(df_600['berita'].tolist())

In [15]:
texts.extend(fact_isi)
labels.extend([1] * len(fact_isi))

texts.extend(hoax_judul)
labels.extend([0] * len(hoax_judul))

texts.extend(hoax_isi)
labels.extend([0] * len(hoax_isi))

In [16]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(texts, labels, test_size = 0.2)
len(train_X), len(test_X)

(34429, 8608)

In [17]:
train_X.extend(df_fake['News'].tolist())
train_Y.extend([0] * len(df_fake['News'].tolist()))

train_X.extend(df_real['News'].tolist())
train_Y.extend([1] * len(df_real['News'].tolist()))

In [18]:
import re
from unidecode import unidecode

alphabets = '([A-Za-z])'
prefixes = (
    '(Mr|St|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|Mt|Puan|puan|Tuan|tuan|sir|Sir)[.]'
)
suffixes = '(Inc|Ltd|Jr|Sr|Co)'
starters = '(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever|Dia|Mereka|Tetapi|Kita|Itu|Ini|Dan|Kami)'
acronyms = '([A-Z][.][A-Z][.](?:[A-Z][.])?)'
websites = '[.](com|net|org|io|gov|me|edu|my)'
another_websites = '(www|http|https)[.]'
digits = '([0-9])'

def split_into_sentences(text):
    text = unidecode(text)
    text = ' ' + text + '  '
    text = text.replace('\n', ' ')
    text = re.sub(prefixes, '\\1<prd>', text)
    text = re.sub(websites, '<prd>\\1', text)
    text = re.sub(another_websites, '\\1<prd>', text)
    if '...' in text:
        text = text.replace('...', '<prd><prd><prd>')
    if 'Ph.D' in text:
        text = text.replace('Ph.D.', 'Ph<prd>D<prd>')
    text = re.sub('\s' + alphabets + '[.] ', ' \\1<prd> ', text)
    text = re.sub(acronyms + ' ' + starters, '\\1<stop> \\2', text)
    text = re.sub(
        alphabets + '[.]' + alphabets + '[.]' + alphabets + '[.]',
        '\\1<prd>\\2<prd>\\3<prd>',
        text,
    )
    text = re.sub(
        alphabets + '[.]' + alphabets + '[.]', '\\1<prd>\\2<prd>', text
    )
    text = re.sub(' ' + suffixes + '[.] ' + starters, ' \\1<stop> \\2', text)
    text = re.sub(' ' + suffixes + '[.]', ' \\1<prd>', text)
    text = re.sub(' ' + alphabets + '[.]', ' \\1<prd>', text)
    text = re.sub(digits + '[.]' + digits, '\\1<prd>\\2', text)
    if '”' in text:
        text = text.replace('.”', '”.')
    if '"' in text:
        text = text.replace('."', '".')
    if '!' in text:
        text = text.replace('!"', '"!')
    if '?' in text:
        text = text.replace('?"', '"?')
    text = text.replace('.', '.<stop>')
    text = text.replace('?', '?<stop>')
    text = text.replace('!', '!<stop>')
    text = text.replace('<prd>', '.')
    sentences = text.split('<stop>')
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences if len(s) > 10]
    return sentences

In [19]:
from albert import tokenization
tokenizer = tokenization.FullTokenizer(
      vocab_file='albert/albert-base-2020-04-10/sp10m.cased.v10.vocab', do_lower_case=False,
      spm_model_file='albert/albert-base-2020-04-10/sp10m.cased.v10.model')


INFO:tensorflow:loading sentence piece model


In [20]:
def augment(string, label, maxlen = 400):
    results, sentences, tokenized = [], '', []
    splitted = split_into_sentences(string)
    for no, s in enumerate(splitted):
        t = tokenizer.tokenize(s)
        if len(tokenized) + len(t) >= maxlen:
            results.append(sentences.strip())
            tokenized = tokenizer.tokenize(splitted[no-1])
            sentences = splitted[no-1]

        sentences += s + ' '
        tokenized.extend(t)
    if len(sentences):
        results.append(sentences.strip())
    return results, [label] * len(results)

In [21]:
augment(train_X[0], train_Y[0])

(['akan membuat keputusan untuk menarik balik sepuluh kali apa yang akan diambil, di luar wikileaks, untuk membawa mesin clinton (tunai) ke bawah akan menang bukit dan kemudian mengumumkan wwiii terhadap russia iran syria "paksi jahat" akan timur tengah benar-benar meletup akan pivot untuk asia benar-benar meletup akan cina akan memerintah dunia menjelang 2025 di tengah-tengah begitu banyak pecahan kegilaan realiti geopolitik yang diwarnakan dengan rapi terhadap runtuhan kami, godaan tidak dapat ditarik kembali kepada tuan jean baudrillard yang terlambat, hebat, dekonstruksi. semasa mod post 1980-an ia menjadi pinggang menjadi inti america nya, yang asalnya diterbitkan dalam perancis pada tahun 1986, masih harus dibaca hari ini sebagai instagram budaya geologi metafisis definitif exceptionalistan. menjelang akhir tahun 1990-an, pada akhir alaf, dua tahun sebelum 9 11, peristiwa "sebelum dan selepas" yang seminalis telah menegaskan bagaimana kita hidup dalam labirin pasaran gelap. Kini,

In [24]:
from tqdm import tqdm

X_train, Y_train = [], []

for i in tqdm(range(len(train_X))):
    x, y = augment(train_X[i], train_Y[i])
    X_train.extend(x)
    Y_train.extend(y)

100%|██████████| 34900/34900 [01:39<00:00, 351.00it/s]


In [26]:
len(X_train), len(Y_train)

(68887, 68887)

In [27]:
from tqdm import tqdm

X_test, Y_test = [], []

for i in tqdm(range(len(test_X))):
    x, y = augment(test_X[i], test_Y[i])
    X_test.extend(x)
    Y_test.extend(y)

100%|██████████| 8608/8608 [00:25<00:00, 332.17it/s]


In [30]:
from sklearn.utils import shuffle

In [31]:
X_train, Y_train = shuffle(X_train, Y_train)

In [32]:
X_test, Y_test = shuffle(X_test, Y_test)

In [33]:
import pickle

with open('relevant-dataset.pkl', 'wb') as fopen:
    pickle.dump({'train_X': X_train, 'train_Y': Y_train,
                'test_X': X_test, 'test_Y': Y_test}, fopen)